In [ ]:
# dataframes
import pandas as pd
import datetime

# read files
import os
import glob 

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# math
import numpy as np
import math

# optimisation 
import pulp
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# spatial data analysis
import geopandas as gpd
from geopy.distance import geodesic
import utm
import pyproj
from shapely.ops import transform
import re
from shapely.geometry import Polygon
from shapely.geometry import MultiPoint 
from shapely.geometry import LineString




#other 
import warnings

# Read Data

In [ ]:
def calculate_incremental_distance(row, df):
    """_summary_

    Args:
        row (_type_): _description_
        df (_type_): _description_

    Returns:
        _type_: _description_
    """
    if row.name == 0: # for the first row, set incremental distance to 0
        return 0
    else:
        # calculate the distance between the current observation and the previous observation
        prev_lat = df.at[row.name - 1, 'Latitude']
        prev_lon = df.at[row.name - 1, 'Longitude']
        curr_lat = row['Latitude']
        curr_lon = row['Longitude']
        prev_coord = (prev_lat, prev_lon)
        curr_coord = (curr_lat, curr_lon)
        return geodesic(prev_coord, curr_coord).kilometers



In [ ]:
def dms_to_decimal(dms_string):
    # Split the DMS string into its parts (degrees, minutes, seconds, direction)
    parts = re.split('[?\'"]+', dms_string)
    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    direction = parts[3]
    
    # Convert the DMS values to decimal
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    if direction in ('S', 'W'):
        decimal *= -1
    
    return decimal

Old Taxi Data

In [ ]:
# Set the working directory
filepath = 'Data/taxi_gps_data/data/'

# Create list of files
files = glob.glob(os.path.join(filepath, '*.csv'))

# Initialize dictionary to store all vehicle trips in {vehicle_id: trip_data}
all_vehicles = {}

# Loop through all files
for file in files:
    # Take extension out of filename
    filename = file.split('.')[0]
    # Take filepath out of filename
    filename = filename.split('/')[-1]
    # Read the trip and clean the data
    this_trip = pd.read_csv(file)
    this_trip['Time'] = pd.to_datetime(this_trip['Time'])
    this_trip['Time_diff'] = this_trip['Time'].diff()
    this_trip['Location'] = list(zip(this_trip['Latitude'], this_trip['Longitude']))
    this_trip['Distance'] = this_trip.apply(calculate_incremental_distance, df = this_trip, axis=1)
    # Save the trip data to a dictionary with the filename as the key 
    all_vehicles[filename] = this_trip
    
    

New GoMetro Data

In [ ]:
def read_csv_files(directory):
    """
    Reads in all CSV files in the specified directory and returns a dictionary of Pandas DataFrames.
    The DataFrame key is generated from the characters after the second "-" in the filename but before ".csv".
    """
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    data_frames = {}
    for file in csv_files:
        print("Processing", file)
        # Extract key from filename
        key = file.split("-")[2].split(".csv")[0].strip()

        # Read CSV file into DataFrame
        path = os.path.join(directory, file)
        df = pd.read_csv(path)
        
        # Check if latitude and longitude columns need to be converted from DMS to decimal
        # if ('Latitude' in df.columns) and ('Longitude' in df.columns):
        #     lat_col = df['Latitude']
        #     lon_col = df['Longitude']
        #     if any(isinstance(val, str) for val in lat_col) and any(isinstance(val, str) for val in lon_col):
        #         # Convert latitude and longitude values from DMS to decimal
        #         df['Latitude'] = df['Latitude'].apply(dms_to_decimal)
        #         df['Longitude'] = df['Longitude'].apply(dms_to_decimal)

        # Add time-related columns and drop unnecessary columns
        df['Time'] = pd.to_datetime(df['Date'] + ' ' + df['Timestamp'])
        df['Time_diff'] = df['Time'].diff()
        df['Distance_calc'] = df.apply(calculate_incremental_distance, df=df, axis=1)
        df.drop(['Date', 'Timestamp'], axis=1, inplace=True)
            
        # Add to dictionary
        data_frames[key] = df
        
    return data_frames


all_vehicles = read_csv_files("Data/Output/")

In [ ]:

for vehicle in all_vehicles:
    odometer = all_vehicles[vehicle]['Distance'].sum()/1e3
    diff = odometer - all_vehicles[vehicle]['Distance_calc'].sum()
    print("vehicle", vehicle, ": ", format((diff/odometer) * 100, ".2f"), "% difference")

In [ ]:
all_vehicles['3.6']

Substation Location Data

In [ ]:
# Read in the substation Shapefile data (input from user)
substation_location_data = gpd.read_file('Data/mv_station_position/MV_STATION_POSITION.shp')
print(substation_location_data.head(5))

In [ ]:
substation_location_data.plot(marker = 'x', color = 'black', label = 'Substations')

# Extract trips and breaks
breaks_array[i] corresponds to the break between trips_array[i] and trips_array[i + 1]


In [ ]:
def extract_breaks_and_trips(gps_data, min_dwell_time, stop_threshold):
    ''' 
    Function to extract the trip and break segments from vehicle GPS data, based on a given minimum dwell time and stop threshold
    '''

    
    # Initialize the trips and breaks lists
    trips = []
    breaks = []
    
    # Assume starts trip moving
    stopped = False
    
    # set starting idx of whatever segment we are on (trip or break)
    this_segment_start_idx = 0 
    this_segment_end_idx = 0
    
    for row in gps_data.itertuples():
        if row.Speed <= stop_threshold: # If the vehicle is stopped
            
            # if the vehicle WAS previously moving -- capture a trip segment and start a break segment
            if not stopped: 
                # set end index of trip to the previous index
                this_segment_end_idx = row.Index - 1 
                # capture trip
                trips.append(gps_data.loc[this_segment_start_idx:this_segment_end_idx]) 
                # reset start index of break to the current index
                this_segment_start_idx = row.Index 
            
            stopped = True
        else:  # If the vehicle is moving
            
            # if the vehicle WAS stopped -- capture a break segment and start a trip segment if the break was long enough. 
            if stopped: 
                # calculate the amount of time the vehicle was stopped
                stopped_time = (row.Time - gps_data.iloc[this_segment_start_idx].Time).total_seconds()
                
                # if break was long enough (convert min_dwell_time from minutes to seconds)
                if stopped_time >= min_dwell_time * 60:  
                    this_segment_end_idx = row.Index - 1
                    breaks.append(gps_data.loc[this_segment_start_idx:this_segment_end_idx])
                    gps_data['charging'].loc[this_segment_start_idx:this_segment_end_idx] = 1
                    this_segment_start_idx = row.Index
                    
                else:#  if the break wasn't long enough then continue logging as a trip
                    pass 
                
            stopped = False 
    # if vehicle is still moving at the end and not already captured, capture the final trip
    if not stopped:
        trips.append(gps_data.loc[this_segment_start_idx:])
        
                
            
    return trips, breaks
            
            
        

In [ ]:
def extract_breaks_and_trips_optimized(gps_data, min_dwell_time, stop_threshold):
    ''' Function to extract the trip and break segments from vehicle GPS data, based on a given minimum dwell time and stop threshold'''
    
    # Initialize the trips and breaks lists
    trips = []
    breaks = []
    # Assume starts trip moving
    stopped = False
    # set starting idx of whatever segment we are on (trip or break)
    this_segment_start_idx = 0 
    this_segment_end_idx = 0
    
    # create a list of tuples containing the indices, speed and times for each GPS data point
    gps_points = [(i, row.Speed, row.Time) for i, row in enumerate(gps_data.itertuples())]
    
    # iterate through the gps_points and group them into trips and breaks
    for i, speed, time in gps_points:
        if speed <= stop_threshold: # If the vehicle is stopped
            if not stopped:   # if the vehicle WAS previously moving -- capture a trip segment and start a break segment
                # set end index of trip to the previous index
                this_segment_end_idx = i - 1 
                # capture trip
                trips.append(gps_data.loc[this_segment_start_idx:this_segment_end_idx])
                # reset start index of break to the current index
                this_segment_start_idx = i 
            stopped = True
        else:  # If the vehicle is moving
            if stopped: # if the vehicle WAS stopped -- capture a break segment and start a trip segment if the break was long enough. 
                # calculate the amount of time the vehicle was stopped
                stopped_time = (time - gps_data.iloc[this_segment_start_idx].Time).total_seconds()
                # if break was long enough (convert min_dwell_time from minutes to seconds)
                if stopped_time/60 >= min_dwell_time:  # convert stopped time from seconds to minutes
                    this_segment_end_idx = i - 1
                    breaks.append(gps_data.loc[this_segment_start_idx:this_segment_end_idx])
                    gps_data['charging'].loc[this_segment_start_idx:this_segment_end_idx] = 1
                    this_segment_start_idx = i
            stopped = False 
        
    # if vehicle is still stopped at the end, capture the final break
    if stopped:
        breaks.append(gps_data.loc[this_segment_start_idx:])
    else:
        # capture the final trip
        trips.append(gps_data.loc[this_segment_start_idx:])

    return trips, breaks



In [ ]:
def find_substations_within_buffer(breaks, substation_location_data, buffer):
    '''
    Function to find substations near a vehicle's break points (to use as candidate charging stations)
    Input: Break data, substation location data, buffer distance
    Output: Array of substation locations within the buffer distance of the break points
    TODO: Integrate into the solution (extra section in case study in the paper?)
    ''' 
    # Find UTM zone based on the average latitude and longitude of the break data
    utm_zone = utm.from_latlon(breaks['Latitude'].mean(), breaks['Longitude'].mean())[2]

    # Define the projection
    project = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:326{utm_zone}")

    # Project the points to the UTM zone
    projected_points = breaks.geometry.apply(lambda p: transform(project.transform, p))

    # Calculate a buffer of 500m around each point in the projected coordinate system
    route_buffer = projected_points.buffer(buffer)

    # Define the inverse projection to use (from UTM to WGS84)zone
    inv_project = pyproj.Transformer.from_crs(f"EPSG:326{utm_zone}", "EPSG:4326")

    # Transform the projected buffer back to EPSG:4326
    buffer_latlon = route_buffer.geometry.apply(lambda p: transform(inv_project.transform, p))

    # Convert the route buffer to a GeoDataFrame 
    route_buffer_gdf = gpd.GeoDataFrame(geometry=buffer_latlon)

    # Dissolve the route buffer to a single polygon (avoids issues of overlapping polygons which are inevitable with a high frequency GPS dataset)
    route_buffer_gdf_dissolved = route_buffer_gdf.dissolve()

    # Merge substations within buffer distance of substations
    substations_within_buffer = gpd.sjoin(substation_location_data, route_buffer_gdf_dissolved, op='within')
    
    # Print number of substations within buffer
    print("There are", len(substations_within_buffer['REC_ID'].unique()), "substations within the buffer")
    
    # # Plot route, route buffer, and substations within buffer
    # route_buffer_gdf.plot(color = 'blue', alpha = 0.1, label = 'Route Buffer')
    # break_locations.plot(ax = plt.gca(), color = 'red', markersize = 0.5, marker = 'o', alpha = 0.5, label = 'Vehicle GPS Locations')
    # substations_within_buffer.plot(ax = plt.gca(), color = 'black', marker = 'x', markersize = 40, label = 'Substations')
    # plt.legend(loc = 'upper right', bbox_to_anchor = (1,1))
    # plt.show()
    
    return substations_within_buffer

In [ ]:
### Retrieve all breaks ### 

# Define stop threshold (km/h) that the vehicle must be moving slower than to be considered stopped
stop_threshold = 1

# Define minimum time (minutes) that the vehicle must be moving slower than stop threshold to be considered on a break
min_dwell_time = 60

# Define average energy consumption (kWh/km) 
kWh_km = 0.50

# Define average charging rating (kW)
charge_rate = 22 

# Implement the extract trips and extract breaks functions for all vehicles
all_trips = {}
all_candidate_nodes = {}

for vehicle in all_vehicles:
    # Make column to identify whether vehicle is charging or not
    all_vehicles[vehicle]['charging'] = 0
    
    print('Extracting trips and breaks from', vehicle)
    trips, breaks = extract_breaks_and_trips_optimized(all_vehicles[vehicle], min_dwell_time, stop_threshold)
    all_trips[vehicle] = trips
    #feasible_substations = find_substations_within_buffer(breaks, substation_location_data, buffer = 500)
    all_candidate_nodes[vehicle] = breaks #+ feasible_substations
    
    # Calculate energy flow in each step based on vehicle status (charging or not)
    all_vehicles[vehicle]['energy_consumed'] = np.where(all_vehicles[vehicle]['charging'] == 0, kWh_km * all_vehicles[vehicle]['Distance']/1e3, 0)
    all_vehicles[vehicle]['energy_charged'] = np.where(all_vehicles[vehicle]['charging'] == 1, charge_rate * all_vehicles[vehicle]['Time_diff'].dt.total_seconds()/3600, 0)
    

    
    print('Vehicle', vehicle, 'travelled', round(all_vehicles[vehicle]['Distance'].sum()/1e3,2),'km with', 
              len(trips), 'trips and', len(breaks), 'breaks and consumed',
              round(all_vehicles[vehicle]['energy_consumed'].sum(),2) ,'kWh and charged',
              round(all_vehicles[vehicle]['energy_charged'].sum(),2), 'kWh \n')




In [ ]:

def get_slow_periods(df):
    # Initialize an empty list to store the periods of slow speed
    slow_periods = []
    start_indices = []
    end_indices = []
    # Loop through the dataframe
    for i in range(len(df)):
        # If the speed is less than 3 km/h, mark the start of a potential slow period
        if df.loc[i, 'Speed'] < 2:
            j = i + 1
            slow_start = df.loc[i, 'Time']
            
            # Continue looping until the speed is greater than or equal to 3 km/h
            while j < len(df) and df.loc[j, 'Speed'] < 3:
                j += 1
            
            # If the slow period lasted at least 60 minutes, save it to the list of slow periods
            if (df.loc[j-1, 'Time'] - slow_start).total_seconds() >= 3600:
                if (j-1) not in end_indices:
                    slow_periods.append(df.loc[i:j-1])
                    start_indices.append(i)
                    end_indices.append(j-1)
    
    # Return a dataframe containing all the slow periods
    return pd.concat(slow_periods), start_indices, end_indices
all_breaks = {}
all_starts = {}
all_ends = {}
polygons ={}
for vehicle in all_vehicles:
    all_breaks[vehicle], all_starts[vehicle], all_ends[vehicle] = get_slow_periods(all_vehicles[vehicle])
    


# Output for GridSim

DBSCAN

In [ ]:


def cluster_locations(df):
    coords = np.radians(df[['Latitude', 'Longitude']])
    scaler = StandardScaler().fit(coords)
    coords_scaled = scaler.transform(coords)
    dbscan = DBSCAN(eps=0.5, min_samples=1).fit(coords_scaled)
    df['Cluster'] = dbscan.labels_
    
    # Plotting 
    clusters = df.groupby('Cluster')
    fig, ax = plt.subplots(figsize=[10, 6])
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'gray', 'brown', 'pink']
    for name, group in clusters:
        if name != -1:
            ax.scatter(group.Longitude, group.Latitude, c=colors[name % len(colors)], label=name, alpha=0.5)
    ax.scatter(df[df.Cluster == -1].Longitude, df[df.Cluster == -1].Latitude, c='black', marker='x', label='Outliers', alpha=0.5)
    ax.legend()
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
   # plt.show()
    return df
dbscan_clusters = {}
for vehicle in all_breaks:
    cluster_labels = cluster_locations(all_breaks[vehicle])
    dbscan_clusters[vehicle] = cluster_labels.groupby('Cluster').mean(['Latitude', 'Longitude'])
    
# output the cluster center longitude and latitudes to a csv file, using the vehicle ID as name of file
for vehicle in dbscan_clusters:
    #mkdir 'ForGridSim' if does not exist
    if not os.path.exists('ForGridSim/dbscan_cluster_centers'):
        os.makedirs('ForGridSim/dbscan_cluster_centers')

    dbscan_clusters[vehicle][['Latitude','Longitude']].to_csv('ForGridSim/dbscan_cluster_centers/dbscan_cluster_centers' + vehicle + '.csv')


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    # Function to calculate the Haversine distance between two lat/lon points in kilometers

    R = 6371  # Radius of the earth in km

    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)

    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c  # Distance in km

    return distance

In [ ]:
def get_top_n_nodes(df, n, min_dist):

    # loop through all the candidate node datafrmes in test_break_set, and save the top 5 nodes with most time spent
    times = []
    for node in df:
        times.append(node['Time_diff'].dt.total_seconds().sum())

    # find the indeces in the times list of the top 5 nodes with most time spent
    sorted_nodes = np.argsort(times)[::-1]#[-30:]
            
    # filtered_nodes = [test_break_set[sorted_nodes[0]]]
    filtered_nodes =[df[sorted_nodes[0]]]
    for node in np.take(df, sorted_nodes[1:], axis=0):
        too_close = False
        for filtered_node in filtered_nodes:
            if haversine(node['Latitude'].mean(), node['Longitude'].mean(),
                        filtered_node['Latitude'].mean(), filtered_node['Longitude'].mean()) < min_dist:
                too_close = True
                break
        if not too_close:
            filtered_nodes.append(node)
        if len(filtered_nodes) == n:
            break

    # get the mean lat and long of the filtered nodes
    locations = []
    for node in filtered_nodes:
        locations.append([node['Latitude'].mean(), node['Longitude'].mean()])

    # calculate pairwise distances between all locations
    distances = []
    for i in range(len(locations)):
        row = []
        for j in range(len(locations)):
            dist = haversine(locations[i][0], locations[i][1], locations[j][0], locations[j][1]) * 1e3 # convert from km to m
            row.append(dist)
        distances.append(row)
    
    times = [] 
    for node in filtered_nodes:
        times.append(node['Time_diff'].dt.total_seconds().sum()/60)

    return distances, locations, times

n = 10
buffer = 0.150 # km 

for vehicle in all_candidate_nodes:
    dist, loc, time = get_top_n_nodes(all_candidate_nodes[vehicle], n, buffer)
    df =pd.DataFrame(loc, columns = ['Latitude', 'Longitude'])
    df['Dwell_time_(mins)'] = time
    for i in range(len(dist)):
        df['Dist_for_loc_' + str(i) + '_(meters)'] = dist[i]
    df.to_csv('ForGridSim/break_period_nodes/' + vehicle + '_nodes.csv')

# Convert break periods into polygons

In [ ]:
def create_charging_area(gdf, buffer_size):

    # Find UTM zone based on the average latitude and longitude of the route
    utm_zone = utm.from_latlon(gdf['Latitude'].mean(), gdf['Longitude'].mean())[2]

    # Define the projection
    project = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:326{utm_zone}")

    # Project the points to the UTM zone
    projected_points = gdf.geometry.apply(lambda p: transform(project.transform, p))

    # Calculate a buffer of size 'buffer_size' (meters) around each point in the projected coordinate system
    buffer = projected_points.buffer(buffer_size)

    # Define the inverse projection to use (from UTM to WGS84)zone
    inv_project = pyproj.Transformer.from_crs(f"EPSG:326{utm_zone}", "EPSG:4326")

    # Transform the projected buffer back to EPSG:4326
    buffer_latlon = buffer.geometry.apply(lambda p: transform(inv_project.transform, p))

    # Convert the buffer to a GeoDataFrame 
    buffer_gdf = gpd.GeoDataFrame(geometry=buffer_latlon)
    

    # Dissolve the route buffer to a single polygon (avoids issues of overlapping polygons which are inevitable with a high frequency GPS dataset)
    buffer_gdf_dissolved = buffer_gdf.dissolve()
    
    # Calculate total dwelling time in this charging area
    buffer_gdf_dissolved['dwell_time'] = (gdf['Time_diff'].dt.total_seconds()/60).sum()

    return buffer_gdf_dissolved





In [ ]:
#### GDFs not working so test with points #### 

# suppress warnings
warnings.filterwarnings("ignore")

buffer_size = 100
charging_areas = {}
for vehicle in all_breaks:
    this_vehicle_charging_areas = []
    for break_period in all_breaks[vehicle]:
    
        # create Points from lat/lon of break periods
        break_period['geometry'] = [Point(xy) for xy in zip(break_period['Longitude'], break_period['Latitude'])]
        
        
        # set the CRS to WGS84 (lat/lon)
        break_period = gpd.GeoDataFrame(break_period, geometry = 'geometry', crs = "EPSG:4326")

        # Get the charging area (overlap of buffered break points) and the dwell time in that area
        charging_area = create_charging_area(break_period, buffer_size)

        # Add the charging area to the list of charging areas for this vehicle
        this_vehicle_charging_areas.append(charging_area)
        
    # Convert the list of charging areas to a GeoDataFrame    
    gdf = gpd.GeoDataFrame(
    {
        'geometry': [poly[0].iloc[0].geometry for poly in this_vehicle_charging_areas],
        'dwell_time': [poly[1] for poly in this_vehicle_charging_areas]
    },
    crs=polygons[0][0].crs # assuming all polygons have the same CRS
    )
    
    # Add the charging areas to the dictionary of charging areas indexed by vehicle
    charging_areas[vehicle] = gdf
    
warnings.filterwarnings("default")

In [ ]:
gdf = charging_areas[vehicle]

sjoin = gpd.sjoin(gdf, gdf)

overlaying_buffers_gdf = sjoin.loc[sjoin.index != sjoin.index_right]
overlaying_buffers_gdf#grr

In [ ]:
def collapse_charging_areas(gdf):
    merged = gpd.GeoDataFrame(geometry=[], crs=gdf.crs)
    for idx, row in gdf.iterrows():
        poly = row['geometry']
        dwell_time = row['dwell_time']
        
        # find all the polygons that intersect with the current polygon
        intersects = gdf[gdf.geometry.intersects(poly)]
        
        # merge the polygons and sum the dwell times
        merged_poly = intersects.geometry.unary_union
        merged_dwell_time = intersects['dwell_time'].sum()
        
        # add the merged polygon to the merged dataframe
        merged = merged.append({'geometry': merged_poly, 'dwell_time': merged_dwell_time}, ignore_index=True)

    # reset the index and convert the 'dwell_time' column to float
    merged = merged.reset_index(drop=True)
    merged['dwell_time'] = merged['dwell_time'].astype(float)

    # only take unique results from the merged dataframe
    unique = merged.drop_duplicates(subset=['geometry'])
    
    return unique

collapsed_charging_areas = {}   

for vehicle in charging_areas:
    collapsed_charging_areas[vehicle] = collapse_charging_areas(charging_areas[vehicle])


In [ ]:
for vehicle in collapsed_charging_areas:
    #sort by dwell time
    collapsed_charging_areas[vehicle] = collapsed_charging_areas[vehicle].sort_values(by='dwell_time', ascending=False)
    # output to file 
    if not os.path.exists('ForGridSim/collapsed_polgyons'):
        os.makedirs('ForGridSim/collapsed_polgyons')
        
    # convert centroid to lat/lon
    collapsed_charging_areas[vehicle]['location'] = collapsed_charging_areas[vehicle].centroid.apply(lambda p: (p.y, p.x))
    
    collapsed_charging_areas[vehicle][['location','dwell_time']].to_csv(f'ForGridSim/collapsed_polgyons/{vehicle}.csv')

In [ ]:
from shapely.ops import cascaded_union

def merge_polygons(df, geom_col):
    # create a new column to store individual polygon IDs
    df['poly_id'] = range(len(df))
    
    # create a list of tuples of each polygon and its ID
    polys = [(poly, id) for poly, id in zip(df[geom_col], df['poly_id'])]
    
    # define an empty list to store intersects groups
    intersect_groups = []
    
    # iterate through each polygon tuple
    for i, (poly_1, id_1) in enumerate(polys):
        # check intersection against all other polygons after the current one
        for j, (poly_2, id_2) in enumerate(polys[i+1:]):
            if poly_1.intersects(poly_2):
                # add both IDs to the intersection group list
                intersect_group = set([id_1, id_2])
                
                # check if either ID is already in another intersect group
                in_group = False
                for group in intersect_groups:
                    if intersect_group.intersection(group):
                        # if so, merge the groups
                        group.update(intersect_group)
                        intersect_group = group
                        in_group = True
                
                # if not, add this intersect group as a new item
                if not in_group:
                    intersect_groups.append(intersect_group)
    
    # iterate through each intersect group and merge their polygons
    merged_polys = []
    for group in intersect_groups:
        group_polys = [polys[id][0] for id in group]
        merged_poly = cascaded_union(group_polys)
        merged_polys.append(merged_poly)
    
    # create a final list of merged polygons and individual polygons that were not merged
    final_polys = []
    for i, (poly, id) in enumerate(polys):
        if id not in set.union(*intersect_groups):
            final_polys.append(poly)
    
    # add the merged polygons to the final list
    final_polys += merged_polys
    
    # convert the final list back to a GeoSeries and return it
    return gpd.GeoSeries(final_polys)
polygons=[]
for vehicle in collapsed_charging_areas:
    merged = merge_polygons(collapsed_charging_areas[vehicle], 'geometry')
    polygons.append(merged)
for polygon in polygons:
    polygon.plot()
# find the centroid of all the polygons and multipolygons in collapsed_charging_areas
for vehicle in collapsed_charging_areas:
    collapsed_charging_areas[vehicle]['centroid'] = collapsed_charging_areas[vehicle]['geometry'].apply(lambda p: p.centroid)
    print(len(collapsed_charging_areas[vehicle]))

# Extract top-n break points for each taxi

In [ ]:
all_dwelling_times = {}

for vehicle in all_candidate_nodes:
    dwelling_times = []
    for break_period in all_candidate_nodes[vehicle]:
        dwelling_times.append(break_period['Time_diff'].dt.total_seconds().sum()/60)
        
    sorted_nodes = np.argsort(dwelling_times)
    
    
    
    
    all_dwelling_times[vehicle] = dwelling_times
    

# loop through all dwelling times for each vehicle, and use the index of the top 5 dwelling times to get the top5 candidate noede
len(all_dwelling_times[vehicle])

In [ ]:
# Create dataframe out of locations and time, where locations is a list of lists of lat and long, and time is a list of times
df =pd.DataFrame(loc, columns = ['Latitude', 'Longitude'])
df['Dwell_time_mins'] = time
# add dist matrix to dataframe (5 new columns)
for i in range(len(dist)):
    df['dist' + str(i)] = dist[i]
df

Tier 0: Finds the top 5 breakpoints for each taxi and how much time spent at each, regardless of how far apart they are from each other

In [ ]:
test_break_set = all_candidate_nodes[vehicle]

# loop through all the candidate node datafrmes in test_break_set, and save the top 5 nodes with most time spent
times = []
for node in test_break_set:
    times.append(node['Time_diff'].dt.total_seconds().sum())
    if node['Time_diff'].dt.total_seconds().sum() < 3600:
        print('too short')

# find the indeces in the times list of the top 5 nodes with most time spent
top_5 = np.argsort(times)[-5:]

# get the top 5 nodes from test_break_set
top_5_nodes = [test_break_set[i] for i in top_5]

# get the mean lat and long of the top 5 nodes
locations = []
for node in top_5_nodes:
    locations.append([node['Latitude'].mean(), node['Longitude'].mean()])


# Calculate pairwise distances between all locations
distances = []
for i in range(len(locations)):
    row = []
    for j in range(len(locations)):
        dist = haversine(locations[i][0], locations[i][1], locations[j][0], locations[j][1]) * 1e3 # convert from km to m
        row.append(dist)
    distances.append(row)
    
# Print the result
for row in distances:
    print(row)
    
# Calculate time spent at each location
dwell_times = []
for node in top_5_nodes:
    print(node['Time_diff'].dt.total_seconds().sum()/3600)




Tier 1: Find the top 5 break points that are min_dist away from each other, and the time spent at each

In [ ]:
min_dist = 0.20 # km

test_break_set = all_candidate_nodes[vehicle]

# loop through all the candidate node datafrmes in test_break_set, and save the top 5 nodes with most time spent
times = []
for node in test_break_set:
    times.append(node['Time_diff'].dt.total_seconds().sum())

# find the indeces in the times list of the top 5 nodes with most time spent
sorted_nodes = np.argsort(times)#[-30:]

filtered_nodes = [test_break_set[sorted_nodes[0]]]
for node in np.take(test_break_set, sorted_nodes[1:], axis=0):
    too_close = False
    for filtered_node in filtered_nodes:
        if haversine(node['Latitude'].mean(), node['Longitude'].mean(),
                     filtered_node['Latitude'].mean(), filtered_node['Longitude'].mean()) < min_dist:
            too_close = True
            break
    if not too_close:
        filtered_nodes.append(node)
    if len(filtered_nodes) == 5:
        break

# get the mean lat and long of the filtered nodes
locations = []
for node in filtered_nodes:
    locations.append([node['Latitude'].mean(), node['Longitude'].mean()])

# calculate pairwise distances between all locations
distances = []
for i in range(len(locations)):
    row = []
    for j in range(len(locations)):
        dist = haversine(locations[i][0], locations[i][1], locations[j][0], locations[j][1]) * 1e3 # convert from km to m
        row.append(dist)
    distances.append(row)

# print the result
for row in distances:
    print(row)
    
for node in filtered_nodes:
    print(node['Time_diff'].dt.total_seconds().sum()/3600)


In [ ]:
# Define maximum battery capacity (kWh)
battery_cap = 70

for vehicle in ['T1001', 'T3001']:
    print("Simulating", vehicle)
    all_vehicles[vehicle]['SOC'] = battery_cap 
       
    for idx, row in all_vehicles[vehicle].iterrows():
        if idx == 0: 
            all_vehicles[vehicle].loc[idx, 'SOC'] = battery_cap # first observation 
        else:
            # State of charge equals minimum of previous state of charge plus energy charged minus energy consumed in this step or battery capacity
            all_vehicles[vehicle].loc[idx, 'SOC'] = np.minimum(all_vehicles[vehicle].loc[idx-1, 'SOC'] + row['energy_charged'] - row['energy_consumed'], battery_cap)
    if (all_vehicles[vehicle]['SOC'] < 0).any():
        print(vehicle, 'ran out of charge!')

In [ ]:
for vehicle in all_vehicles:
    plt.plot(all_vehicles[vehicle].SOC)
    plt.ylim(0,70)
    plt.show()

In [ ]:
from math import radians, sin, cos, sqrt, asin

def find_close_points(df):
    # Convert latitude and longitude from degrees to radians
    df['lat_rad'] = df['latitude'].apply(radians)
    df['lon_rad'] = df['longitude'].apply(radians)

    close_points = []
    for i, row_i in df.iterrows():
        for j, row_j in df.iterrows():
            if i >= j:
                continue  # skip self-matches and duplicates
            lat1, lon1, lat2, lon2 = row_i['lat_rad'], row_i['lon_rad'], \
                                     row_j['lat_rad'], row_j['lon_rad']
            # Compute distance between two points using haversine formula
            dlon = lon2 - lon1 
            dlat = lat2 - lat1 
            a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
            c = 2 * asin(sqrt(a)) 
            r = 6371 # Radius of earth in kilometers. Use 3956 for miles
            dist = c * r
            
            # Check if distance is less than 500m (0.5km)
            if dist < 0.500:
                close_points.append((i,j))
    
    return close_points


In [ ]:
# run the simulation
a = datetime.now()
Sim = Simulator(df_array_breaks, df_array_trips, 1.0)
Sim.run_simulation()
b = datetime.now()
print(b-a)

In [ ]:
class Vehicle:
    """_summary_line_
    Class to simulate a vehicle with a battery and a route
    """
    # initiate all the variables
    def __init__(self, initial_distance, service_time, vehicle_params, params):
        self.capacity = vehicle_params['Capacity'] # kWh
        self.consumption = vehicle_params['kWh_km']  # kWh/km
        self.charge_steps = vehicle_params['charge_steps'] # kWh
        self.charge_rate = vehicle_params['charge_rate'] # list of charging rate at each charge step (kWh/min)
        self.break_10 = 0.1 * self.capacity
        self.break_5 = 0.05 * self.capacity
        self.break_70 = 0.7 * self.capacity
        self.distance = initial_distance
        self.service_time = service_time
        self.total_time_trips = 0.
        self.km_so_far = 0.
        self.time_break = 0.
        self.min_so_far = 0.
        self.trips = True
        self.breaks = False
        self.charger = False
        self.was_charging = False
        self.end_charging = False
        self.indec_cs = 0
        self.sim_breaks = {"Duration": [], "Longitude": [], "Latitude": [], "Order": [], "Charged": [], "Charging Duration": [],
                          "Initial minute": [], "Initial SoC": []}
        self.sim_trips = {"Distance": [], "Order": []}
        self.order = 0
        self.charged = 0.
        self.duration = 0.
        self.in_m = 0.
        self.in_soc = 52.
        self.wasted_time = 0.
        self.travel_time = 0.
        self.not_charging = 0.
        # stochastic parameters
        self.lambda_dist = params[0]
        self.mu_speed = params[1]
        self.sigma_speed = params[2]
        self.lambda_speed = params[3]
        self.omega_speed = params[4]
        self.mu_breaks = params[5]
        self.sigma_breaks = params[6]
        
    # set a random wasted time when drivers charge
    def set_wasted_time(self, mu=2., sigma=0.33):
        return np.random.normal(loc=mu, scale=sigma)
    
    # simulate charging time
    def charge(self, duration):
        init_cap = float(self.capacity)
        for s in range(len(self.charge_steps)):
            if self.capacity < self.charge_steps[s] and duration > 0.:
                mins = self.decide_duration(duration, self.charge_steps[s] - self.capacity, s)
                self.capacity += mins * self.charge_rate[s]
                duration -= mins
        self.charged += self.capacity - init_cap
        return
    
    # energy consumption per distance
    def calculate_energy(self, distance):
        return distance * self.consumption
    
    
        # function used to change between trip and break or viceversa. In both cases, all variables need to be reset
    def change_state(self, t):
        if self.capacity < 0.:
            self.capacity = 0.
        if self.trips:
            self.trips = False
            self.sim_trips["Distance"].append(self.km_so_far)
            self.sim_trips['Order'].append(self.order)
            self.order += 1
            self.breaks = True
            self.time_break = np.random.lognormal(self.mu_breaks, self.sigma_breaks)
            location = np.random.randint(0, df_total_breaks.shape[0])
            while df_total_breaks["Inside City"].values[location] == False:
                location = np.random.randint(0, df_total_breaks.shape[0])
            self.sim_breaks["Duration"].append(self.time_break)
            self.sim_breaks["Longitude"].append(df_total_breaks["Longitude"].values[location])
            self.sim_breaks['Latitude'].append(df_total_breaks["Latitude"].values[location])
            self.sim_breaks['Order'].append(self.order)
            self.in_m = float(t / 60.)
            self.in_soc = float(self.capacity)
            self.charged = 0.
            self.duration = 0.
            self.order += 1
            self.min_so_far = 0.
            if (self.time_break >= 30. and self.capacity <= self.break_70) or self.capacity <= self.break_10:
                self.charger = True
                self.was_charging = True
        else:
            if self.total_time_trips < t * self.service_time / 100.:
                self.trips = True
                self.breaks = False
                self.sim_breaks['Initial minute'].append(self.in_m)
                self.sim_breaks['Initial SoC'].append(self.in_soc)
                if self.duration > self.time_break:
                    self.duration = self.time_break
                if self.charged + self.in_soc > 52.:
                    self.charged = 52. - self.in_soc
                self.duration -= self.wasted_time
                if self.duration < 0.:
                    self.duration = 0.
                    self.charged = 0.
                    self.capacity = float(self.in_soc)
                self.sim_breaks['Charged'].append(self.charged)
                self.sim_breaks['Charging Duration'].append(self.duration)
                self.distance = np.random.exponential(1. / self.lambda_dist)
                self.km_so_far = 0.
                self.wasted_time = 0.
                self.travel_time = 0.
                self.not_charging = 0.
            else:
                tot_time = self.total_time_trips / (self.service_time / 100.)
                self.time_break += (tot_time - t) / 60.
                
                
    # at each interval, change capacity based on what driver is doing
    def update_state(self, t):
        if self.trips:
            # based on statistical distributions, set the speed the driver is using in the interval of time 
            # and reduce capacity accordingly
            stops = np.random.uniform(0.0, 1.0, size=6)
            l = 6 - len(stops[stops < 0.21752482333083625])
            speed = self.omega_speed * np.random.normal(loc=self.mu_speed, scale=self.sigma_speed, size=l) \
                        + (1. - self.omega_speed) * np.random.exponential(1. / self.lambda_speed, size=l)
            distance_t = np.sum(speed) / (3.6 * 1000.) * 10.0
            self.km_so_far += distance_t
            self.capacity -= self.calculate_energy(distance_t)
            self.total_time_trips += 60.
        if self.breaks:
            # if in a break, just wait for next interval of time or increase capacity if driver is charging
            if self.travel_time > 0.:
                self.travel_time -= 1.
            else:
                self.min_so_far += 1.
                if self.not_charging > 0.:
                    self.not_charging -= 1.
                else:
                    if self.was_charging:
                        self.charge(1.)
                    if (self.capacity >= 52. or self.min_so_far >= self.time_break) and self.was_charging:
                        self.duration = float(self.min_so_far)
                        self.was_charging = False
                        self.end_charging = True
                        if self.capacity > 52.:
                            self.capacity = 52.
        
        # if break/trip has come to an end, change state
        if (self.trips and self.km_so_far >= self.distance) or (self.breaks and self.min_so_far >= self.time_break):
            self.change_state(t)
    

# Simple numerical optimisation examples
Which breaks to charge at <br>
Next step: which _locations_ to charge at <br>
Step after that: Redundancy constraint <br>
BUT before that... I want to make the sure the energy consumption column makes sense... [not necessary to go super deep on this, because it's not the point of this paper. Can use ev_fleet_sim or whatever to simulate the energy consumption]

In [ ]:
# array of energy consumed along each trip
# to do this one, i may have to rework the extract trips and breaks function so that consecutive trips are appended together
Ec = []
for trip in all_trips[vehicle]:
    Ec.append(trip['energy_consumed'].sum())
# array of energy recharged at each break
Er = []
for candidate_node in all_candidate_nodes[vehicle]:
    Er.append(candidate_node['energy_charged'].sum())

Plotting SOC simulation as sanity check

In [ ]:
# simulation of vehicle to make sure it is possible
# Define maximum battery capacity (kWh)
BC = 100

for vehicle in ['3.6', '3.7']:
    print("Simulating", vehicle)
    all_vehicles[vehicle]['SOC'] = BC 
       
    for idx, row in all_vehicles[vehicle].iterrows():
        if idx == 0: 
            all_vehicles[vehicle].loc[idx, 'SOC'] = BC # first observation 
        else:
            # State of charge equals minimum of previous state of charge plus energy charged minus energy consumed in this step or battery capacity
            all_vehicles[vehicle].loc[idx, 'SOC'] = np.minimum(all_vehicles[vehicle].loc[idx-1, 'SOC'] + row['energy_charged'] - row['energy_consumed'], BC)
    if (all_vehicles[vehicle]['SOC'] < 0).any():
        print(vehicle, 'ran out of charge!')
        
    # plot the results
    plt.plot(all_vehicles[vehicle]['SOC'])
    plt.show()

In [ ]:

    
def identify_charging_opportunities(gps_data, min_dwell_time, stop_threshold):
    ''' Function to identify charging opportunities in the gps data'''
    
    gps_data['charging'] = False
    
    # Assume starts trip moving
    charging = False
    # set starting idx of whatever segment we are on (trip or break)
    this_segment_start_idx = 0 
    this_segment_end_idx = 0
    
    # create a list of tuples containing the indices, speed and times for each GPS data point
    gps_points = [(i, row.Speed, row.Time) for i, row in enumerate(gps_data.itertuples())]
    
    # iterate through the gps_points and group them into trips and breaks
    for i, speed, time in gps_points:
        if speed <= stop_threshold: # If the vehicle is stopped
            if not stopped:   # if the vehicle WAS previously moving -- capture a trip segment and start a break segment
                # set end index of trip to the previous index
                this_segment_end_idx = i - 1 
                # capture trip
                trips.append(gps_data.loc[this_segment_start_idx:this_segment_end_idx])
                # reset start index of break to the current index
                this_segment_start_idx = i 
            stopped = True
        else:  # If the vehicle is moving
            if stopped: # if the vehicle WAS stopped -- capture a break segment and start a trip segment if the break was long enough. 
                # calculate the amount of time the vehicle was stopped
                stopped_time = (time - gps_data.iloc[this_segment_start_idx].Time).total_seconds()
                # if break was long enough (convert min_dwell_time from minutes to seconds)
                if stopped_time/60 >= min_dwell_time:  # convert stopped time from seconds to minutes
                    this_segment_end_idx = i - 1
                    breaks.append(gps_data.loc[this_segment_start_idx:this_segment_end_idx])
                    gps_data['charging'].loc[this_segment_start_idx:this_segment_end_idx] = 1
                    this_segment_start_idx = i
            stopped = False 
        
    # if vehicle is still stopped at the end, capture the final break
    if stopped:
        breaks.append(gps_data.loc[this_segment_start_idx:])
    else:
        # capture the final trip
        trips.append(gps_data.loc[this_segment_start_idx:])

    return trips, breaks



In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def find_charging_opportunities(df):
    """
    Iterates through GPS data and creates a new column called 'charging_opportunity',
    which = 1 for chunks of consecutive data where the vehicle is at less than 2km/h for
    at least 60 minutes, and 0 otherwise.
    
    Parameters:
    df (pd.DataFrame): a DataFrame with datetime and speed columns
    
    Returns:
    pd.DataFrame: the original DataFrame with a new column 'charging_opportunity'
    """
    
    # Create a timedelta object for 60 minutes
    one_hour = timedelta(minutes=60)
    
    # Create a new column 'charging_opportunity' and initialize it to 0
    df['charging_opportunity'] = 0
    
    # Loop through the DataFrame and mark chunks of consecutive data where the vehicle
    # is at less than 2km/h for at least 60 minutes as charging opportunities
    start_time = None
    for i, row in df.iterrows():
        # If the speed is less than 2km/h and we haven't started a new charging opportunity yet,
        # record the start time
        if row['Speed'] < 1 and start_time is None:
            start_time = row['Time']
        # If the Speed is greater than 2km/h or we've reached the end of the DataFrame, 
        # and we've already recorded a start time, check if the duration is long enough
        elif (row['Speed'] >= 1 or i == len(df)-1) and start_time is not None:
            duration = row['Time'] - start_time
            if duration >= one_hour:
                # Mark the charging opportunity as 1 for this chunk of data
                df.loc[(df['Time'] >= start_time) & (df['Time'] < row['Time']), 'charging_opportunity'] = 1
            # Reset the start time
            start_time = None
    
    return df
copy_df = all_vehicles['3.6'].copy()
copy_df = find_charging_opportunities(copy_df)

(copy_df['charging_opportunity'] - copy_df['charging']).sum()


In [ ]:
avg_kWh_km = 0.53 # kWh/km
charge_rate = 22 # kW 


df = all_vehicles['3.6']

# create a new column 'group' to identify consecutive sequences of 'charging'
df['group'] = (df['charging'] != df['charging'].shift()).cumsum()

# group the dataframe by 'group' and apply sum aggregation
trip_segments = df.groupby('group').agg({
    'charging': 'mean', 
    'Distance': 'sum', 
    'Time_diff': 'sum'})
trip_segments
trip_segments['energy_consumed'] = trip_segments['Distance']/1e3 * avg_kWh_km
trip_segments['energy_charged'] = trip_segments['Time_diff'].dt.total_seconds()/3600 * charge_rate
# sanity check that the vehicle wouldn't be consuming loads of energy while it's supposed to be charging! 
# trip_segments['energy_consumed'][trip_segments['charging'] ==1].sum() 
trip_segments['Time_diff_hrs'] = trip_segments['Time_diff'].dt.total_seconds()/3600
trip_segments.groupby('charging').sum()

In [ ]:
# group the dataframe by 'group' and apply sum aggregation to create trip segments of either charging or discharging energy
trip_segments = df.groupby('group').agg({
    'charging': 'mean',
    'Distance': 'sum',
    'Latitude': 'mean',
    'Longitude': 'mean',
    'Time_diff': 'sum'})
trip_segments

In [ ]:
# plot trip segments latitude and longitude and color the dots by the charging column
trip_segments['charging_status'] = np.where(trip_segments['charging'] == 1, 'blue', 'red')
plt.scatter(trip_segments['Longitude'], trip_segments['Latitude'], c=trip_segments['charging_status'])



In [ ]:
from scipy.spatial.distance import cdist

# create list of candidate location coordinates
CNL = []

for break_point in all_candidate_nodes[vehicle]:
    CNL.append((break_point['Longitude'].mean(), break_point['Latitude'].mean()))
               
# Define a threshold distance for group membership
threshold = 150  # meters

# Compute pairwise distances between all points in CNL
distances = cdist(CNL, CNL)

# Create an adjacency matrix based on distances below threshold
adjacency = distances < threshold

# Use depth-first search to find connected components
visited = set()
CNG = [None] * len(CNL)
group_index = 0

def dfs(i):
    CNG[i] = group_index
    visited.add(i)
    neighbors = [j for j in range(len(CNL)) if adjacency[i, j] and j not in visited]
    for j in neighbors:
        dfs(j)

for i in range(len(CNL)):
    if i not in visited:
        dfs(i)
        group_index += 1


In [ ]:
from pulp import *
avg_kWh_km = 0.53 # kWh/km
charge_rate = 22 # kW 


df = all_vehicles['3.6']

# create a new column 'group' to identify consecutive sequences of 'charging'
df['group'] = (df['charging'] != df['charging'].shift()).cumsum()

# group the dataframe by 'group' and apply sum aggregation to create trip segments of either charging or discharging energy
trip_segments = df.groupby('group').agg({
    'charging': 'mean',
    'Distance': 'sum',
    'Location': 'first',
    'Time_diff': 'sum'})

trip_segments['energy_consumed'] = trip_segments['Distance'] / \
    1e3 * avg_kWh_km  # m/1e3 * kWh/km = kWh
trip_segments['energy_charged'] = trip_segments['Time_diff'].dt.total_seconds(
)/3600 * charge_rate  # hrs * kW = kWh


BC = 100  # kWh
SOC_max = 1  # %
SOC_in = SOC_max
SOC_min = 0
S_start = 0

# amount of energy consumed during this trip segment
e_c = trip_segments['energy_consumed'][trip_segments['charging'] == 0].values
# amount of energy charged during this trip segment
e_r = trip_segments['energy_charged'][trip_segments['charging'] == 1].values
if len(e_r) < len(e_c):
    # add an element to the end of e_r that takest he battery back up to 100
    e_r = np.append(e_r, 100 - e_c[-1])
elif len(e_c) < len(e_r):
    # add an element to the end of e_c to make them even in length
    e_c = np.append(e_c, 0)

# amount of energy at each trip segment
e = [-e_c[i] + e_r[i] for i in range(len(e_c))]

# Sanity check that none of the trip segments are too long
for val in e:
    if val < (-BC * SOC_max):
        raise ValueError(
            f"One of the trip segments requires {val} kWh, which is more than the battery capactiy allows.")

# Define candidate_nodes as a list of integers representing the indices of the stops where charging stations can be built
candidate_nodes = range(len(e_r))  # indeces of candidate nodes

n = len(e_c)

### Define optimization problem
model = LpProblem("EV charging optimization modell", LpMinimize)

### Define decision variables
X = LpVariable.dicts("UseLocation", range(
    len(e)), lowBound=0, upBound=1, cat=LpBinary)
E = LpVariable.dicts("Energy", range(len(e)), lowBound=0,
                     upBound=100, cat=LpContinuous)


### Define objective function - minimize number of charging stations
model += lpSum(X[i] for i in candidate_nodes)

### Constraints

# Initial energy level
model += E[S_start] == SOC_in * BC


for i in range(n-1):
    # Energy at next stop (only charge if charging station built there)
    model += E[i+1] <= E[i] - e_c[i] + (e_r[i] * X[i])
    # Energy does not exceed battery capacity
    model += E[i+1] <= BC * SOC_max
    # Has enough energy to reach the next stop without going below minimum safe battery level
    model += E[i] >= e_c[i] + (SOC_min * BC)


# Solve
status = model.solve()

# print the "build plan":
build_plan = [i for i in candidate_nodes if X[i].varValue > 0]
print('build at nodes: ')
print(build_plan)

# print the sum of the energy at each node
sum1 = sum([E[i].varValue for i in candidate_nodes])

plt.plot(candidate_nodes, [
         E[i].varValue for i in candidate_nodes], color='r', label='no bonus')
plt.scatter(build_plan, [0 for t in build_plan], s=10,
            alpha=0.6, color='orange', label='no-bonus charge')

# replace the objective function to include a bonus for carrying more energy. This ensures the vehicle charges up as much as possible at each node. The bonus is small enough that it doesn't outweight the cost of building anew station, so we use a weighting factor of 1/(BC * SOC_Max * num_nodes) which would be the bonus for having a full charge  at 100 nodes
bonus = BC * SOC_max * len(candidate_nodes)
model += lpSum(X[i] for i in candidate_nodes) - 1 / \
    (bonus)*lpSum(E[i] for i in candidate_nodes)
status2 = model.solve()

# QA the build plan has the same NUMBER of build points
build_plan_2 = [i for i in candidate_nodes if X[i].varValue > 0]
# assert len(build_plan) == len(build_plan_2)
print(len(build_plan), len(build_plan_2))

print('build at nodes: ', build_plan_2)
plt.plot(candidate_nodes, [
         E[i].varValue for i in candidate_nodes], color='b', label='bonus')
plt.scatter(build_plan_2, [0 for t in build_plan_2],
            s=10, alpha=0.6, color='g', label='charge')
plt.legend(loc='upper right')
plt.show()
# print the sum of the energy at each node
sum2 = sum([E[i].varValue for i in candidate_nodes])


In [ ]:
sum1, sum2